# __2. 데이터와 표본분포__

##### 파이썬 패키지, 데이터 import

In [ ]:
%matplotlib inline

from pathlib import Path

import pandas as pd
import numpy as np
from scipy.stats import trim_mean
from statsmodels import robust
import wquantiles

import seaborn as sns
import matplotlib.pylab as plt

In [ ]:
state = pd.read_csv('./data/state.csv')
dfw = pd.read_csv('./data/dfw_airline.csv')
sp500_px = pd.read_csv('./data/sp500_data.csv.gz', index_col=0)
sp500_sym = pd.read_csv('./data/sp500_sectors.csv')
kc_tax = pd.read_csv('./data/kc_tax.csv.gz')
lc_loans = pd.read_csv('./data/lc_loans.csv')
airline_stats = pd.read_csv('./data/airline_stats.csv')

## __2.1 임의표본추출과 표본편향__

#### __표본__ : 더 큰 데이터 집합(모집단)으로부터 얻은 부분집합
#### __모집단__ : 어떤 데이터 집합을 구성하는 전체 대항 혹은 전체 집합 (이론적인 가상의 집합을 의미하기도 함)
#### __N__ : 모집단(표본)의 크기

#### __임의표본추출(임의표집, 랜덤표본추출)__ : 모집단 내 선택 가능한 원소들을 무작위로 표본 추출
#### __단순임의표본__ : 임의표본추출로 얻은 표본
#### 복원추출 : 중복 추출이 가능하도록 해당 샘플을 다시 모집단에 포함시킴
#### 비복원추출 : 한 번 뽑힌 원소는 추후 추첨에 사용하지 않음

#### __층화표본추출(층화표집)__ : 모집단을 층으로 나눈 뒤, 각 층에서 무작위로 표본을 추출하는 것
#### __계층__ : 공통된 특징을 가진 모집단의 동종 하위 그룹

#### __대표성__ : 완결성, 형식의 일관성, 깨끗함 및 각 데이터 값의 정확성

#### __편향(bias)__ : 측정 과정 혹은 표본추출과정에서 발생하는 계통적인(systematic) 오차
#### __표본편항__ : 모집단을 잘못 대표하는 표본
#### 대표되도록 의도된 모집단으로부터 추출되지 않고 비임의(nonrandom) 방식으로 표본이 추출

## __2.2 선택편향__

#### __선택편향__ : 관측 데이터를 선택하는 방식 때문에 생기는 편향
#### 데이터 스누핑 : 뭔가 흥미로운 것을 찾아 광범위하게 데이터를 살피는 것
#### 방대한 검색 효과 : 중복 데이터 모델링이나 너무 많은 예측변수를 고려하는 모델링에서 비롯되는 편향 혹은 비재현성